## Deconvolution Test


In [13]:
from s2cnn import S2Convolution

import math
import sys
import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from scipy import spatial

import torch
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

from average_meter import AverageMeter
from data_source import DataSource
from data_splitter import DataSplitter
from database_parser import DatabaseParser

from loss import ImprovedTripletLoss
from mission_indices import MissionIndices
from model import Model
from sphere import Sphere
from training_set import TrainingSet
from visualize import Visualize
from model_encode_decode_simple import ModelEncodeDecodeSimple

from torch.autograd import Variable
from torch.utils.tensorboard import SummaryWriter
from s2cnn import so3_near_identity_grid, S2Convolution, s2_near_identity_grid, SO3Convolution, so3_integrate

from tqdm.auto import tqdm

%matplotlib inline
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
#torch.cuda.set_device(0)
#torch.backends.cudnn.benchmark = True

bandwidth = 30
restore = False
n_epochs = 55
batch_size = 13
num_workers = 32
descriptor_size = 256
net_input_size = 2 * bandwidth
n_features = 3
cache = 1
n_data = 52

#net = ModelEncodeDecodeSimple(bandwidth).cuda()
net = ModelEncodeDecodeSimple(bandwidth)

In [3]:
dataset_path = "/mnt/data/datasets/Spherical/test_training_data/"
db_parser = DatabaseParser(dataset_path)

ds = DataSource(dataset_path, cache)
ds.load(n_data)

Reading missions db from /mnt/data/datasets/Spherical/test_training_data/missions.csv
Read    mission_anchor mission_positive mission_negative
0      6725..0000       6725..0000       6725..0000
1      6725..0000       6725..0000       6725..0000
2      4e53..0000       7799..0000       4e53..0000
3      4e53..0000       7799..0000       4e53..0000
4      4e53..0000       7799..0000       4e53..0000
5      4e53..0000       7799..0000       4e53..0000
6      4e53..0000       7799..0000       4e53..0000
7      4e53..0000       7799..0000       4e53..0000
8      4e53..0000       7799..0000       4e53..0000
9      4e53..0000       7799..0000       4e53..0000
10     4e53..0000       7799..0000       4e53..0000
11     4e53..0000       7799..0000       4e53..0000
12     4e53..0000       7799..0000       4e53..0000
13     4e53..0000       7799..0000       4e53..0000
14     4e53..0000       7799..0000       4e53..0000
15     4e53..0000       7799..0000       4e53..0000
16     4e53..0000       7


Loading positives from:	/mnt/data/datasets/Spherical/test_training_data//training_positive_pointclouds/ and /mnt/data/datasets/Spherical/test_training_data//training_positive_sph_images/



Loading negatives from:	/mnt/data/datasets/Spherical/test_training_data//training_negative_pointclouds/ and /mnt/data/datasets/Spherical/test_training_data//training_negative_sph_images/



Done loading dataset.
	Anchor point clouds total: 	52
	Anchor images total: 		52
	Anchor poses total: 		52
	Positive point clouds total: 	52
	Positive images total: 		52
	Positive poses total: 		52
	Negative point clouds total: 	52
	Negative images total: 		52
	Negative poses total: 		52


In [5]:
train_set = TrainingSet(restore, bandwidth)
train_set.generateAll(ds)

Generating anchor spheres



Generating positive spheres



Generating negative spheres



Generated features
Generating features from 0 to 52


In [6]:
print("Total size: ", len(train_set))
split = DataSplitter(train_set, restore, test_train_split=1.0, shuffle=True)

train_loader, val_loader, test_loader = split.get_split(
    batch_size=batch_size, num_workers=num_workers)
train_size = split.get_train_size()
val_size = split.get_val_size()
test_size = split.get_test_size()
print("Training size: ", train_size)
print("Validation size: ", val_size)
if test_size == 0:
    print('Test size is 0. Configured for external tests')
else:
    print("Testing size: ", test_size)


Total size:  52
Training size:  46
Validation size:  6
Test size is 0. Configured for external tests


In [15]:
for batch_idx, (data1, data2, data3) in enumerate(train_loader):
    #data1, data2, data3 = data1.cuda().float(), data2.cuda().float(), data3.cuda().float()      
    data1, data2, data3 = data1.float(), data2.float(), data3.float()
    break

In [16]:
print(f"input shape: {data1.shape}")
embedded_a = net(data1)
print(f"computed embedding: {embedded_a.shape}")

input shape: torch.Size([13, 2, 60, 60])
computed embedding: torch.Size([13, 5, 20, 20, 20])
